In [1]:
import fxcmpy
import socketio
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sympy import *
init_printing()
import xlrd
import itertools
import datetime
from itertools import combinations
from itertools import product
from itertools import groupby
from astropy.table import QTable, Table, Column
from astropy import units as u
from IPython.display import HTML
import requests
import json

C:\ProgramData\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
TOKEN = 'b199bb5a15fa26957990a3fb823a51bd473cca55'
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error', server='demo', log_file='log.txt')


## WATCHLIST = ['AUD/CAD','AUD/CHF','AUD/JPY','AUD/NZD','AUD/USD','CAD/CHF','CAD/JPY','CHF/JPY','EUR/AUD','EUR/CAD','EUR/CHF','EUR/GBP','EUR/JPY','EUR/NZD','EUR/USD','GBP/AUD','GBP/CAD','GBP/CHF','GBP/JPY','GBP/USD','NZD/CAD','NZD/CHF','NZD/JPY','NZD/USD','USD/CAD','USD/CHF','USD/JPY']

pair = ['AUD/CAD','AUD/CHF','AUD/JPY','AUD/NZD','AUD/USD','CAD/CHF','CAD/JPY','CHF/JPY','EUR/AUD','EUR/CAD','EUR/CHF','EUR/GBP','EUR/JPY','EUR/NZD','EUR/USD','GBP/AUD','GBP/CAD','GBP/CHF','GBP/JPY','GBP/USD','NZD/CAD','NZD/CHF','NZD/JPY','NZD/USD','USD/CAD','USD/CHF','USD/JPY']
GRADE = []
MOVE = []

h = len(pair)
index = np.arange(0,h)
gradetable = pd.DataFrame(index=index, columns=['PAIR','MOVE','GRADE'])

for x in pair:
    S = con.get_candles(x, period='H4', number=1920)          ## SETTING TO 12 DAYS FOR NOW
    S['open'] = (S['bidopen']+S['askopen'])/2
    S['close'] = (S['bidclose']+S['askclose'])/2
    S['high'] = (S['bidhigh']+S['askhigh'])/2
    S['low'] = (S['bidlow']+S['asklow'])/2
    S = S.assign(MOVE=(S['close'] - S['open']))
    AvgCandle = np.average(abs(S['MOVE']))

    s = 96                                      ## NUMBER OF 1-HOUR CANDLESTICKS WE WANT TO FORECAST AHEAD (4 DAYS)

    CP = S['close'].iloc[-1]
    l = int(S['close'].count())
    Sd = np.linspace(1,l,num=l)

    D = con.get_candles(x, period='H4', number=1920)          ## SETTING TO 12 DAYS FOR NOW
    D['open'] = (D['bidopen']+D['askopen'])/2
    D['close'] = (D['bidclose']+D['askclose'])/2
    D['high'] = (D['bidhigh']+D['askhigh'])/2
    D['low'] = (D['bidlow']+D['asklow'])/2
    D = D.assign(MOVE=(D['close'] - D['open']))
    AvgCandleH4 = np.average(abs(D['MOVE']))

    k = int(D['close'].count())
    Sdk = np.linspace(1,k,num=k)


## TREND ANALYSIS SECTOR


    M = []
    H = []
    G = []
    a = l
    b = 0

    for q in Sd:
        x = np.linspace(1, a, num=a)
        y = S.iloc[b:,10]
    
        m = sm.OLS(y, x).fit()
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        m = slope
        h = intercept
        g = m*(a+s)+h
    
        M.append(m)
        H.append(h)
        G.append(g)
        
        a = a - 1
        b = b + 1


    w = int(round(l*(14/16),0))
    trendnow = np.average(M[w:l-1])
        

    w1 = int(round(l*(1/16),0))
    M_long1 = np.average(M[w1:l-1])
    w2 = int(round(l*(3/16),0))
    M_long2 = np.average(M[w2:l-1])
    w3 = int(round(l*(6/16),0))
    M_long3 = np.average(M[w3:l-1])
    Tgrade_long = (0.15*M_long1 + 0.35*M_long2 + 0.50*M_long3)
    if trendnow > 0:
        if Tgrade_long > 0:
            Tgrade_long = Tgrade_long
        else:
            Tgrade_long = 0
    elif trendnow < 0:
        if Tgrade_long > 0:
            Tgrade_long = 0
        else:
            Tgrade_long = -1*Tgrade_long
    else:
        Tgrade_long = 0


    w1 = int(round(l*(8/16),0))
    M_mid1 = np.average(M[w1:l-1])
    w2 = int(round(l*(10/16),0))
    M_mid2 = np.average(M[w2:l-1])
    w3 = int(round(l*(12/16),0))
    M_mid3 = np.average(M[w3:l-1])
    Tgrade_mid = (0.20*M_mid1 + 0.4*M_mid2 + 0.4*M_mid3)
    if trendnow > 0:
        if Tgrade_mid > 0:
            Tgrade_mid = Tgrade_mid
        else:
            Tgrade_mid = 0
    elif trendnow < 0:
        if Tgrade_mid > 0:
            Tgrade_mid = 0
        else:
            Tgrade_mid = -1*Tgrade_mid
    else:
        Tgrade_mid = 0


    w1 = int(round(l*(13/16),0))
    M_short1 = np.average(M[w1:l-1])
    w2 = int(round(l*(14/16),0))
    M_short2 = np.average(M[w2:l-1])
    w3 = int(round(l*(15/16),0))
    M_short3 = np.average(M[w3:l-1])
    Tgrade_short = (0.3*M_short1 + 0.3*M_short2 + 0.4*M_short3)
    if trendnow > 0:
        if Tgrade_short > 0:
            Tgrade_short = Tgrade_short
        else:
            Tgrade_short = 0
    elif trendnow < 0:
        if Tgrade_short > 0:
            Tgrade_short = 0
        else:
            Tgrade_short = -1*Tgrade_short
    else:
        Tgrade_short = 0


    wlong = 0.15
    wmid = 0.35
    wshort = 0.50

    Grade = (wlong*Tgrade_long) + (wmid*Tgrade_mid) + (wshort*Tgrade_short)


    if trendnow > 0:
        move = 'rise'
    else:
        move = 'fall'

    GRADE.append(Grade)
    MOVE.append(move)
    
pair = list(pair)
GRADE = list(GRADE)
MOVE = list(MOVE)

gradetable = gradetable.assign(PAIR=pair)
gradetable = gradetable.assign(MOVE=MOVE)
gradetable = gradetable.assign(GRADE=GRADE)

gradetable = gradetable.sort_values(['GRADE'], ascending=False)
gradetable.to_excel('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Monitor/ScreenerLOBF/ScreenerLOBF.xlsx')

G = zip(pair,MOVE,GRADE)
G = list(G)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:140: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:142: RuntimeWarning: invalid value encountered in double_scalar

[('AUD/CAD', 'rise', 0.00022763506940287452), ('AUD/CHF', 'rise', 0.0001746150771483194), ('AUD/JPY', 'rise', 0.010472687907422985), ('AUD/NZD', 'rise', 0.00021560610288917938), ('AUD/USD', 'rise', 0.0001378786062338336), ('CAD/CHF', 'rise', 4.268375445166158e-05), ('CAD/JPY', 'fall', 0.00926970864121816), ('CHF/JPY', 'fall', 0.015659732459778263), ('EUR/AUD', 'fall', 0.0004851122236399821), ('EUR/CAD', 'fall', 7.129655029638066e-05), ('EUR/CHF', 'rise', 1.7789259914192676e-05), ('EUR/GBP', 'fall', 4.476912858062772e-05), ('EUR/JPY', 'fall', 0.016371125847697118), ('EUR/NZD', 'fall', 0.00017470408528611354), ('EUR/USD', 'fall', 6.914725548040637e-05), ('GBP/AUD', 'fall', 0.0004517622265523301), ('GBP/CAD', 'rise', 3.994066895280103e-05), ('GBP/CHF', 'rise', 7.910908017565139e-05), ('GBP/JPY', 'fall', 0.0125740411622122), ('GBP/USD', 'rise', 1.4201211044307821e-05), ('NZD/CAD', 'rise', 4.5977938845929775e-05), ('NZD/CHF', 'rise', 6.535267728906319e-05), ('NZD/JPY', 'rise', 0.00034505912